# PolyfitController Weights Optimization

The `PolyfitController` is an algorithm that controls the platform in the simulation. This algorithm works by sampling worm positions in previous time stamps (previous observations), fitting a polynomial for the worm positions as function of time. A polynomial of a specified degree is fitted, such that the errors between the polynomial value and the observed positions is minimal. Afterwards, the fitted polynomial is sampled in a future time to predict worm's future position. That process is repeated every cycle. 

There are two main parameters for that algorithm:
1.  The degree of the polynomial that is being fitted.
2.  The weight of each observation for the fitting process. The error of an observation is multiplied by it's corresponding weight during the fitting process. 
    Therefore, samples with lower weight contribute less to the fitting process.

In this notebook we seek to find the optimal degree and weight parameters for the algorithm. To this end, we run an optimization algorithm over the search space of weights, and return the best weight found w.r.t the evaluation error. In addition, there is an option to choose the degree of the fitted polynomial, and assess the performance of the best weights for each polynomial degree.
The evaluation error is calculated over an experiment log file (bboxes.csv).

In [ ]:
# fix imports
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
import mealpy
import numpy as np

from wtracker.sim.sim_controllers.polyfit_controller import WeightEvaluator, PolyfitConfig
from wtracker.sim.config import TimingConfig, ExperimentConfig
from wtracker.utils.gui_utils import UserPrompt
from wtracker.utils.path_utils import join_paths

### Experiment configuration and Polyfit algorithm parameters

In [ ]:
from pprint import pprint

################################ User Input ################################

# The path to the experiment folder on which the evaluation is to be performed.
# If None, a file dialog will be shown to select the folder.
experiment_folder = None

############################################################################

if experiment_folder is None:
    experiment_folder = UserPrompt.open_directory(title="Select the experiment folder")

time_config_file = join_paths(experiment_folder, "time_config.json")
exp_config_file = join_paths(experiment_folder, "exp_config.json")
bboxes_file = join_paths(experiment_folder, "bboxes.csv")

print("Time config path: ", time_config_file)
print("Experiment config path: ", exp_config_file)
print("Experiment log file path: ", bboxes_file)

In [ ]:
# load the time and the experiment config

time_config = TimingConfig.load_json(time_config_file)
exp_config = ExperimentConfig.load_json(exp_config_file)

pprint(time_config)
pprint(exp_config)

In [ ]:
################################ User Input ################################

# the degree of the polynomial to be fitted over the data
poly_degree = 1

# The time stamps of the data points to be used for the polynomial fit.
# 0 is the begging of the current cycle, while negative values correspond to previous cycles.
input_offsets = np.asanyarray(
    [
        -3 * time_config.cycle_frame_num + 0,
        -3 * time_config.cycle_frame_num + 6,
        -2 * time_config.cycle_frame_num + 0,
        -2 * time_config.cycle_frame_num + 6,
        -time_config.cycle_frame_num + 0,
        -time_config.cycle_frame_num + 6,
        0,
        3,
    ]
)

############################################################################

### Define the optimization problem

In [ ]:
start_times = np.arange(exp_config.num_frames // time_config.cycle_frame_num) * time_config.cycle_frame_num


# create the evaluator which calculates the evaluation loss for a given set of weights
evaluator = WeightEvaluator(
    csv_path=bboxes_file,
    timing_config=time_config,
    input_offsets=input_offsets,
    start_times=start_times,
    eval_offset=time_config.cycle_frame_num + time_config.imaging_frame_num // 2,
)


# define the evaluation function that receives the weights as input and returns the evaluation loss.
# the optimization process is performed over this function, such that the returned value is minimized.
def eval_func(weights: np.ndarray) -> float:
    return evaluator.eval(weights, deg=poly_degree)

In [ ]:
from mealpy.utils.problem import Problem
from mealpy.utils.termination import Termination
from mealpy.utils.agent import Agent
import mealpy


################################ User Input ################################

# create the optimizer which minimizes the evaluation loss
# feel free to choose any optimizer from the mealpy package
optim = mealpy.PSO.OriginalPSO()

# the termination parameters for the optimization process
termination = Termination(
    max_epoch=300,  # maximum number of iterations
    max_fe=None,  # maximum number of function evaluations
    max_early_stop=100,  # maximum number of iterations without improvement before stopping
)

############################################################################

# define the bounds for the weights, and create the optimization problem
bounds = mealpy.FloatVar(lb=np.zeros(len(input_offsets)), ub=np.ones(len(input_offsets)))
problem = Problem(obj_func=eval_func, bounds=bounds, minimax="min")

### Run the weights optimizer

Note, that running the optimizer and finding the optimal weights might take a while, up to 15 minutes.

In [ ]:
# Run the optimizer on the optimization problem
# Feel free to manually stop the execution of this cell and continue to the next cell, if you are satisfied with the results.
best: Agent = None
best = optim.solve(problem, termination=termination)

In [ ]:
np.set_printoptions(precision=4, suppress=True)

if best is None:
    # If the optimization process is stopped manually, the best solution is the global best solution found so far.
    best = optim.g_best

print("Lowest evaluation loss: ", best.target.fitness)
print("Optimal Weights: ", best.solution / np.linalg.norm(best.solution))

# create the polynomial config object with the best solution found
poly_config = PolyfitConfig(
    degree=poly_degree,
    sample_times=evaluator.input_offsets.tolist(),
    weights=best.solution.tolist(),
)

In [ ]:
################################ User Input ################################

# path to the file where the polynomial config is to be saved
# If None, a file dialog will be shown to select the save path.
polyfit_config_save_path = None

############################################################################

# save the polynomial config
poly_config.save_json(polyfit_config_save_path)